Agent VG multi record producers

In [2]:
# LIBS
import requests
import os
import json
import time
from dotenv import load_dotenv

In [3]:
# LOAD env vars
load_dotenv()

True

In [7]:
# REST API (ServiceNow) URL and credentials
instance = os.getenv('em_lab_instance')
user = os.getenv('em_lab_u_name')
pwd = os.getenv('em_lab_p_word')
url = f"https://{instance}.service-now.com/api/now/table/incident"

# HEADERS
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}

In [9]:
# PING (testing connection)
ping_response = requests.get(url, auth=(user, pwd), headers=headers)
print(ping_response.status_code)

200


In [ ]:
# PAYLOAD GENERATOR
#### Add AI here and payload generator

In [ ]:
# PAYLOADS (temp)
payloads = [
    {
        "short_description": "Demo Incident 1",
        "description": "This is a demo incident 1 for testing purposes.",
        "urgency": "2",
        "impact": "2"
    },
    {
        "short_description": "Demo Incident 2",
        "description": "This is a demo incident 2 for testing purposes.",
        "urgency": "1",
        "impact": "1"
    },
    {
        "short_description": "Demo Incident 3",
        "description": "This is a demo incident 3 for testing purposes.",
        "urgency": "3",
        "impact": "3"
    },
    {
        "short_description": "Demo Incident 4",
        "description": "This is a demo incident 4 for testing purposes.",
        "urgency": "1",
        "impact": "2"
    }
]

In [ ]:
# CREATE INCIDENTs
def create_incident(payload):
    try:
        response = requests.post(url, auth=(user, pwd), headers=headers, data=json.dumps(payload))
        # Check if the response was successful
        if response.status_code == 201:
            print(f"Incident created successfully: {json.loads(response.text)['result']['number']}")
        else:
            print(f"Failed to create incident: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"An error occurred: {e}")

# intevals (simlated INCs arrival)
intervals = [10, 60, 180]  # 10 seconds, 1 minute, 3 minutes

# loop through each payload
for i, payload in enumerate(payloads):
    create_incident(payload)  # create the incident
    if i < len(intervals):  # if it's not the last one, wait for the next interval
        time.sleep(intervals[i])  # wait before sending the next request

| END OF PROGRAM |
|-|